In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import numpy as np
import re
import keras
from tqdm import tqdm
import pickle as pkl
from tensorflow.keras import layers
import tensorflow as tf
import gc
from sklearn.metrics import accuracy_score
import os
gc.enable()

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, TimeDistributed, Activation, Masking, Input
from tensorflow.keras.layers import LSTM, Bidirectional, dot, concatenate
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import * 
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.backend import squeeze
from tensorflow.keras. optimizers import RMSprop, Adam

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
keras.backend.set_session(sess)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10590221345816099547
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4654365438363155553
physical_device_desc: "device: XLA_CPU device"
]
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



##Классификация последовательностей

In [ ]:
order_features = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/order_features.csv')
order_features['prod_seq'] = order_features['prod_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))
order_features['prev_seq'] = order_features['prev_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))
order_features['prev_orders_seq'] = order_features['prev_orders_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))

train_positive = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/train_positive.csv')
train_negative = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/train_negative.csv')

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/id_to_token.pkl', 'rb') as f:
  id_to_token = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/token_to_id.pkl', 'rb') as f:
  token_to_id = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/val_orders.pkl', 'rb') as f:
  val_orders = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/test_orders.pkl', 'rb') as f:
  test_orders = pkl.load(f)

In [ ]:
it = 1
#os.makedirs('/content/drive/My Drive/Colab Notebooks/Diplom/models/simple_lstm/{0}'.format(it))

### Собираем выборки

In [ ]:
pos_neg = pd.concat((train_positive, train_negative), axis=0)

In [ ]:
data_train = pos_neg[(~pos_neg['order_id'].isin(val_orders)) & (~pos_neg['order_id'].isin(test_orders))].sample(frac=1).drop_duplicates(subset=['uxp_total_bought', 'uxp_reorder_ratio',
       'times_lastN', 'u_total_orders', 'u_reordered_ratio', 'p_total_purchases', 'p_reorder_ratio', 'order_dow',
        'order_hour_of_day', 'days_since_prior_order'])
data_val = pos_neg[pos_neg['order_id'].isin(val_orders)].sample(frac=1)
data_test = pos_neg[pos_neg['order_id'].isin(test_orders)].sample(frac=1)

In [ ]:
new_data_train = data_train.merge(order_features[['order_id', 'prev_seq']], how='inner', on='order_id')
new_data_val = data_val.merge(order_features[['order_id', 'prev_seq']], how='inner', on='order_id')
new_data_test = data_test.merge(order_features[['order_id', 'prev_seq']], how='inner', on='order_id')

prods_df = train_positive.merge(order_features[['order_id', 'prev_seq']], how='inner', on='order_id')

In [ ]:
new_data_train['set'] = new_data_train['prev_seq'] + new_data_train['product_id'].apply(lambda x: [x])
new_data_val['set'] = new_data_val['prev_seq'] + new_data_val['product_id'].apply(lambda x: [x])
new_data_test['set'] = new_data_test['prev_seq'] + new_data_test['product_id'].apply(lambda x: [x])

new_data_train['lens'] = new_data_train['set'].apply(len)
new_data_val['lens'] = new_data_val['prev_seq'].apply(len)
new_data_test['lens'] = new_data_test['prev_seq'].apply(len)

In [ ]:
new_val_set = new_data_val[new_data_val['label'] == 1]
new_test_set = new_data_test[new_data_test['label'] == 1]

### Объявляем модель и обучаем

In [ ]:
hidden_size = 10

inp = Input(shape=(None,))

emb = Embedding(len(id_to_token), output_dim=13, name='embedding')(inp)

lstm = LSTM(hidden_size, kernel_initializer='he_uniform', name='lstm_1')(emb)

#target_vec = lstm[:,-1,:]
#query = Dense(hidden_size, kernel_initializer='he_uniform', name='query')(target_vec)

#attention = dot([query, lstm[:,:-1,:]], axes=[1, 2], name='score_attention')
#attention = Activation('softmax', name='attention_activation')(attention)

#context = dot([attention, lstm[:,:-1,:]], axes=[1,1], name='context')
#decoder_combined_context = concatenate([target_vec, context], name='attention_vectors')

output = Dense(hidden_size, activation='tanh', kernel_initializer='he_uniform', name='dense_tanh')(lstm)

out = Dense(2, kernel_initializer='he_uniform', activation='softmax')(output)

model = Model(inp, out)

optimizer = Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['binary_accuracy'])

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 13)          322634    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                960       
_________________________________________________________________
dense_tanh (Dense)           (None, 10)                110       
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 323,726
Trainable params: 323,726
Non-trainabl

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/Diplom/models/simple_lstm/{0}/'.format(it)

In [ ]:
model.load_weights(path+'simple_lstm_lr4_epochs:5_precision:0.250_trainloss:0.684_trainacc:0.554_valloss:0.686_valacc:0.559.hdf5')

In [ ]:
def my_learning_rate(epoch, lrate):
  return lrate

checkpoint = ModelCheckpoint(path + '2cl_lstm_10emb_epochs:{epoch:03d}.hdf5', 
                             monitor='binary_accuracy', verbose=1, save_best_only=True, mode='max', save_freq='epoch')
stop = EarlyStopping(monitor='loss', min_delta=0.0001)
lrs = LearningRateScheduler(my_learning_rate)

callbacks_list = [checkpoint, lrs]

metrics = {}

In [ ]:
batch_size = 64
ep = 10

for ep in range(ep, ep+1):

  print('\n')
  print('Training_epoch_{0}\n'.format(ep))

  train_loss = 0
  train_acc = 0

  train_steps = 0

  for ln, group_l in tqdm(new_data_train.groupby('lens'), position=0):
    temp_train = group_l.sample(frac=1,random_state=432)[['set', 'label']].values

    if batch_size > temp_train.shape[0]:
      batch_size = temp_train.shape[0]

    X_train_seq = np.zeros((temp_train.shape[0], ln), dtype=int)
    y_train = np.zeros((temp_train.shape[0], 2), dtype=int)

    for i in range(temp_train.shape[0]):
      seq = temp_train[i, 0]
      label = temp_train[i, -1]
      for j in range(ln):
        X_train_seq[i,j] = token_to_id[seq[j]]
      y_train[i, label] = 1
    
    history = model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=1, shuffle=False, workers=-1)

    train_loss += history.history['loss'][0]
    train_acc += history.history['binary_accuracy'][0]
    train_steps += 1

  train_loss /= train_steps
  train_acc /= train_steps

  print('\n')
  print('train_loss: ', train_loss)
  print('train_acc: ', train_acc)

  print('\n')
  print('Validation_epoch_{0}\n'.format(ep))

  val_loss = 0
  val_acc = 0

  val_steps = 0

  for ln, group_l in tqdm(new_data_val.groupby('lens'), position=0):
    temp_val = group_l.sample(frac=1,random_state=432)[['set', 'label']].values

    if batch_size > temp_val.shape[0]:
      batch_size = temp_val.shape[0]

    X_val_seq = np.zeros((temp_val.shape[0], ln), dtype=int)
    y_val = np.zeros((temp_val.shape[0], 2), dtype=int)

    for i in range(temp_val.shape[0]):
      seq = temp_val[i, 0]
      label = temp_val[i, -1]
      for j in range(ln):
        X_val_seq[i,j] = token_to_id[seq[j]]
      y_val[i, label] = 1

    loss, acc = model.evaluate(X_val_seq, y_val, batch_size=batch_size, workers=-1)

    val_loss += loss
    val_acc += acc
    val_steps += 1

  val_loss /= val_steps
  val_acc /= val_steps

  print('\n')
  print('val_loss: ', val_loss)
  print('val_acc: ', val_acc)

  print('\n')
  print('Target_metric\n')

  precision_lstm = {}
  recall_lstm = {}
  for i in range(1, 11):
    precision_lstm[i] = 0
    recall_lstm[i] = 0

  total = 0

  for user_id, group_u in tqdm(new_val_set.groupby('user_id'), position=0):

    all_prod_set = prods_df[prods_df['user_id']==user_id].drop_duplicates(subset='product_id')
    all_prod_set['product_id'] = all_prod_set['product_id'].apply(lambda x: token_to_id[x])
    prod_set_features = all_prod_set['product_id'].values

    for order_id, group_o in group_u.groupby('order_id'):
      prod_seq = list(map(lambda x: token_to_id[x], group_o['prev_seq'].values[0]))
      #Купленные товары
      bought = np.array(list(set(group_o['product_id'])))

      #Ищем товары, которые порекомендует сетка
      inp_seq = np.zeros((prod_set_features.shape[0], len(prod_seq) + 1), dtype=int)

      for i in range(prod_set_features.shape[0]):
        temp_arr = prod_seq + [prod_set_features[i]]
        for j in range(len(prod_seq) + 1):
          inp_seq[i, j] = temp_arr[j]

      preds = model.predict(inp_seq)[:, 1]
      indexes = np.argsort(preds)[::-1]

      for i in range(1, 11):
        recommended_lstm = list(map(lambda x: id_to_token[x], prod_set_features[indexes[:i]]))
        precision_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(recommended_lstm)
        recall_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(bought)

      total += 1

  for i in range(1, 11):
    precision_lstm[i] = precision_lstm[i]/total
    recall_lstm[i] = recall_lstm[i]/total

  model.save(path + 'simple_lstm_lr4_epochs:{}_precision:{:.3f}_trainloss:{:.3f}_\
trainacc:{:.3f}_valloss:{:.3f}_valacc:{:.3f}.hdf5'.format(ep, precision_lstm[1], train_loss, train_acc, val_loss, val_acc))
  
  print('\n')
  print(precision_lstm)
  print(recall_lstm)
  print('\n')

###Проверяем качество на тесте

In [ ]:
precision_lstm = {}
recall_lstm = {}
for i in range(1, 11):
  precision_lstm[i] = 0
  recall_lstm[i] = 0

total = 0

for user_id, group_u in tqdm(new_test_set.groupby('user_id'), position=0):
  
  all_prod_set = prods_df[prods_df['user_id']==user_id].drop_duplicates(subset='product_id')
  all_prod_set['product_id'] = all_prod_set['product_id'].apply(lambda x: token_to_id[x])
  prod_set_features = all_prod_set['product_id'].values
  
  for order_id, group_o in group_u.groupby('order_id'):
    prod_seq = list(map(lambda x: token_to_id[x], group_o['prev_seq'].values[0]))
    #Купленные товары
    bought = np.array(list(set(group_o['product_id'])))
    
    #Ищем товары, которые порекомендует сетка
    inp_seq = np.zeros((prod_set_features.shape[0], len(prod_seq) + 1), dtype=int)
    
    for i in range(prod_set_features.shape[0]):
      temp_arr = prod_seq + [prod_set_features[i]]
      for j in range(len(prod_seq) + 1):
        inp_seq[i, j] = temp_arr[j]
        
    preds = model.predict(inp_seq)[:, 1]
    indexes = np.argsort(preds)[::-1]

    for i in range(1, 11):
      recommended_lstm = list(map(lambda x: id_to_token[x], prod_set_features[indexes[:i]]))
      precision_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(recommended_lstm)
      recall_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(bought)

    total += 1

for i in range(1, 11):
  precision_lstm[i] = precision_lstm[i]/total
  recall_lstm[i] = recall_lstm[i]/total

metrics[it] = [precision_lstm, recall_lstm]
  
with open(path + 'simple_lstm_metrics.pkl', 'wb') as f:
  pkl.dump(metrics, f)

print('\n')
print(precision_lstm)
print(recall_lstm)
print('\n')

100%|██████████| 6605/6605 [02:26<00:00, 45.03it/s]
